# 自転車犯罪マップ

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/crime map.png" width=600>

このラボの目的:

- オープンデータソースからデータを取得し、Python ノートブックにインポート
- 複数の列から単一の住所フィールドを作成
- 緯度と経度の座標を割り当てて各行をジオコーディング
- 美しいチャートや地図を作成

## ライブラリをインポートする

このラボで使うライブラリーを一気にインポートしよう。

In [1]:
## for spatial analysis
import geopandas as gpd

## for data analysis
import pandas as pd

## for pretty charts
import plotly.express as px

# for plotly themes
import plotly.io as pio

## for URL requests
import urllib.request
import requests

## for maps
import folium
from folium import plugins

# オープンデータとは？

どんなプロジェクトでも、どんなマップでもデータが必要です。でも「いい」データって意外となかったりするので、イメージしていたマップが作れないことがよくある。そこで、最近政府機関がオープンデータを提供する方向性があり、あらゆる行政がCSVやEXCELフォーマットでデータをダウンロードできるように提供している。

Let's find some open data!

Here is an example:

https://www.pref.chiba.lg.jp/shoufuku/opendata/techoutoukei.html

## 自転車盗難データをダウンロード

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Chiba%20police.png" width=400>

まずは千葉県警察のサイトにアクセス。そこから次に手順でデータをダウンロード：

➡️ https://www.police.pref.chiba.jp/

➡️ 安全な暮らし

➡️ 地域の防犯

➡️ あなたの町の犯罪情勢

➡️ オープンデータ

➡️ 自転車盗（CSV）

ダウンロードしたファイルを `chibike.csv` と名付けて、このフォルダー（Week08) にセーブ。

In [2]:
# ダウンロードしたデータを読み込もう
df = pd.read_csv('chibike.csv', encoding='cp932')

In [3]:
# データの情報
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6317 entries, 0 to 6316
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   罪名             6317 non-null   object 
 1   手口             6317 non-null   object 
 2   管轄警察署（発生地）     6317 non-null   object 
 3   管轄交番・駐在所（発生地）  6314 non-null   object 
 4   市区町村コード（発生地）   6316 non-null   float64
 5   都道府県（発生地）      6316 non-null   object 
 6   市区町村（発生地）      6316 non-null   object 
 7   町丁目（発生地）       6301 non-null   object 
 8   発生年月日（始期）      6317 non-null   object 
 9   発生時（始期）        6317 non-null   object 
 10  発生場所           6317 non-null   object 
 11  発生場所の詳細        6317 non-null   object 
 12  被害者の年齢         6317 non-null   object 
 13  被害者の職業         6317 non-null   object 
 14  施錠関係           6317 non-null   object 
dtypes: float64(1), object(14)
memory usage: 740.4+ KB


Wow! ７千件もある！ちょっと多いので場所で絞りましょう。<h1>「柏」</h1>だけのデータフレームを作ろう。

In [4]:
df

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係
0,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011.0,千葉県,千葉市中央区,新宿２丁目,2021-01-04,03,４階建て以上共同住宅,駐車（輪）場,40歳代,その他,施錠した
1,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011.0,千葉県,千葉市中央区,新宿２丁目,2021-02-15,19,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,30歳代,その他,施錠した
2,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011.0,千葉県,千葉市中央区,新宿２丁目,2021-04-07,20,４階建て以上共同住宅,駐車（輪）場,30歳代,その他,施錠した
3,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011.0,千葉県,千葉市中央区,新宿２丁目,2021-06-17,18,４階建て以上共同住宅,駐車（輪）場,10歳代,その他,施錠した
4,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011.0,千葉県,千葉市中央区,新田町,2021-10-14,16,４階建て以上共同住宅,駐車（輪）場,20歳代,その他,施錠した
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6312,窃盗,自転車盗,四街道,旭交番,122289.0,千葉県,四街道市,小名木,2021-08-31,08,一戸建住宅,駐車（輪）場,30歳代,その他,施錠した
6313,窃盗,自転車盗,四街道,旭交番,122289.0,千葉県,四街道市,鹿渡,2021-04-13,18,一戸建住宅,駐車（輪）場,10歳代,高校生,施錠せず
6314,窃盗,自転車盗,四街道,旭交番,122289.0,千葉県,四街道市,鹿渡,2021-09-29,16,その他の住宅（３階建て以下共同住宅等）,その他,20歳代,その他,施錠した
6315,窃盗,自転車盗,四街道,旭交番,122289.0,千葉県,四街道市,みそら３丁目,2021-09-29,16,その他,その他,40歳代,その他,施錠した


In [5]:
kashiwa = df[df['管轄警察署（発生地）'] == '柏'].copy()

In [11]:
# データをチェック
kashiwa.sample(1)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係
2668,窃盗,自転車盗,柏,南柏駅前交番,122173.0,千葉県,柏市,豊四季,2021-04-11,18,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,20歳代,その他,施錠した


In [7]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 2577 to 2996
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   罪名             420 non-null    object 
 1   手口             420 non-null    object 
 2   管轄警察署（発生地）     420 non-null    object 
 3   管轄交番・駐在所（発生地）  420 non-null    object 
 4   市区町村コード（発生地）   420 non-null    float64
 5   都道府県（発生地）      420 non-null    object 
 6   市区町村（発生地）      420 non-null    object 
 7   町丁目（発生地）       420 non-null    object 
 8   発生年月日（始期）      420 non-null    object 
 9   発生時（始期）        420 non-null    object 
 10  発生場所           420 non-null    object 
 11  発生場所の詳細        420 non-null    object 
 12  被害者の年齢         420 non-null    object 
 13  被害者の職業         420 non-null    object 
 14  施錠関係           420 non-null    object 
dtypes: float64(1), object(14)
memory usage: 52.5+ KB


### `value_counts`でチャート

このデータを見て、咄嗟に知りたいものってなんでしょう？

例えば、<h1>「1日の何時に自転車の盗難が一番発生するの？」</h1>の質問に答えるためにはどのようなデータ分析が必要でしょうか？

では、そのチャートを作ってみましょう。

データには **【発生時（始期）】** のカラムがあるので、各時間帯のカウントを`value_counts()`で調べる。最後に足される`reset_index()`で結果をデータフレームに変換します。

In [23]:
# create a new variable with hourly counts
time = kashiwa['被害者の年齢'].value_counts().reset_index()

time

,被害者の年齢,count
0,10歳代,156
1,20歳代,113
2,30歳代,52
3,40歳代,31
4,50歳代,24
5,70歳以上,18
6,65-69歳,9
7,60-64歳,7
8,法人・団体、被害者なし,6
9,10歳未満,4


この結果は良いが、カラム名（ヘッダー）を直す必要がある。

In [24]:
# fix headers
time.columns = ['被害者の年齢','件数']
time

,被害者の年齢,件数
0,10歳代,156
1,20歳代,113
2,30歳代,52
3,40歳代,31
4,50歳代,24
5,70歳以上,18
6,65-69歳,9
7,60-64歳,7
8,法人・団体、被害者なし,6
9,10歳未満,4


いよいよ準備万端。この新しいデータフレームでチャートを作ろう。この場合は plotly express の bar charts を参考。

https://plotly.com/python/bar-charts/

In [25]:
fig = px.bar(time,x='被害者の年齢',y='件数')
fig.show()

ありゃ？順番が件数の多い順になってる。実はx軸のオプションっていっぱいあるんだ！その中の一つでカテゴリーの順番を設定できる。

https://plotly.com/python/categorical-axes/

In [15]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数'
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

チャートのテンプレートを変えることでルックスが変わる。

`template`のオプションはこちらから：

```["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white"]```

一つずつ試してみよう。

In [16]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='ggplot2' # change this to see other styles
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

### Make your own charts

では、ここで他のチャートを作ってみよう。

例：被害者の年齢、被害者の職業

In [17]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 2577 to 2996
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   罪名             420 non-null    object 
 1   手口             420 non-null    object 
 2   管轄警察署（発生地）     420 non-null    object 
 3   管轄交番・駐在所（発生地）  420 non-null    object 
 4   市区町村コード（発生地）   420 non-null    float64
 5   都道府県（発生地）      420 non-null    object 
 6   市区町村（発生地）      420 non-null    object 
 7   町丁目（発生地）       420 non-null    object 
 8   発生年月日（始期）      420 non-null    object 
 9   発生時（始期）        420 non-null    object 
 10  発生場所           420 non-null    object 
 11  発生場所の詳細        420 non-null    object 
 12  被害者の年齢         420 non-null    object 
 13  被害者の職業         420 non-null    object 
 14  施錠関係           420 non-null    object 
dtypes: float64(1), object(14)
memory usage: 52.5+ KB


In [19]:
# create a new variable with hourly counts
work = kashiwa['被害者の職業'].value_counts().reset_index()

work

,被害者の職業,count
0,その他,231
1,高校生,80
2,大学生,52
3,中学生,39
4,小学生,12
5,法人・団体、被害者なし,6


In [20]:
# fix headers
work.columns = ['被害者の職業','件数']
work

,被害者の職業,件数
0,その他,231
1,高校生,80
2,大学生,52
3,中学生,39
4,小学生,12
5,法人・団体、被害者なし,6


In [21]:
fig = px.bar(work,x='被害者の職業',y='件数')
fig.show()

## その他のチャート（上級編）

複数の変数でチャートを作ると、より深い分析ができる。

In [27]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='施錠関係',
            template='seaborn',
            barmode='group')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

↑上のチャートの凡例ののカテゴリーをダブルクリックするとどうなる？

`barmode='group'`を足すと何がどのように変わる？

In [28]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='施錠関係',
            barmode='group', # group the categories,
            template='seaborn'
            )
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

# Python Lesson: Functions 関数

突然だが、ちょっとここでPythonのレッスンを投入！次のステップでFunction (関数)を使うので、まずは簡単に学ぼう。

関数とは、特定のタスクを実行するために組織化され<span style="font-size:2em">【再利用可能】</span>なコードのブロックです。例えば、繰り返し同じ作業をするぞ！と分かったら、同じコードブロックをコピペするよりかは関数を作った方が良い。


In [29]:
# example of a simple function
def hello():
    print('こんにちは😀')

In [30]:
hello()

こんにちは😀


In [ ]:
# call the function
hello()

In [31]:
# example of a function with an argument
def hello(name):
    print(f'✨こんにちは {name}✨!')

In [33]:
# call the function
hello('Saki')

✨こんにちは Saki✨!


In [34]:
def add(a,b):
    print(a+b)

In [35]:
add(100,55)

155


## チャレンジ問題

メジャーリーグの野球の試合でピッチャーの時速が画面で表示されるが、これは大体MPH (miles per hour)なので、日本人には分かりにくい。

<img src="https://raw.githubusercontent.com/yohman/24-1-Reitaku-GIS/229177aa00887ceccb2a6d0cf2b0cd1263298cc4/Weeks/Week08/images/mlb%20mpg.jpeg" width=600>

では、このMilesをKilometersに変える関数を書きましょう！計算は：

`1 miles = 1.60934 kilometers`

In [50]:
# create the function
def mtokm(miles):
    km = miles * 1.60934
    return km


In [51]:
# call the function
mtokm(104)

167.37136

# Geocoding

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Geocoding_01.png" width=400>

住所だけではマップイングできません。座標が必要です。なので、住所から座標を特定するプロセスが必要である。このプロセスを<h1>【ジオコーティング】</h1>という。


ジオコーティングと言えば、色んな方法があります。現在、日本で無料でジオコーティングサービスを提供しているのが国土地理院のジオコーティングAPI。

試してみよう。このようにURLをブラウザーで記入するだけで座標が返ってくるサービスである。国土地理院さん、とても便利なサービス、ありがとうございます！

https://msearch.gsi.go.jp/address-search/AddressSearch?q=麗澤大学

では、このプロセスに従って、アドレスから座標を出力する Python 関数を作成します。

In [52]:
# 関数を作成
def geocode(address):

    # ジオコーティングURL
    url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

    # try/exceptでエラーをキャッチ
    # 成功の場合
    try:
        s_quote = urllib.parse.quote(address)
        response = requests.get(url + s_quote)
        if len(response.json())>0:
            return response.json()[0]["geometry"]["coordinates"] 
        else:
            return False
        
    # 失敗の場合
    except:
        return False

関数の使い方は簡単！

In [57]:
geocode('東京')

[141.363617, 43.076111]

## データフレームの準備

`kashiwa`のデータフレームの各行の住所をジオコーティングする前に`住所`と座標（`lat`,`lon`）のフィールドを作りましょう。

In [58]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 2577 to 2996
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   罪名             420 non-null    object 
 1   手口             420 non-null    object 
 2   管轄警察署（発生地）     420 non-null    object 
 3   管轄交番・駐在所（発生地）  420 non-null    object 
 4   市区町村コード（発生地）   420 non-null    float64
 5   都道府県（発生地）      420 non-null    object 
 6   市区町村（発生地）      420 non-null    object 
 7   町丁目（発生地）       420 non-null    object 
 8   発生年月日（始期）      420 non-null    object 
 9   発生時（始期）        420 non-null    object 
 10  発生場所           420 non-null    object 
 11  発生場所の詳細        420 non-null    object 
 12  被害者の年齢         420 non-null    object 
 13  被害者の職業         420 non-null    object 
 14  施錠関係           420 non-null    object 
dtypes: float64(1), object(14)
memory usage: 52.5+ KB


In [59]:
# 空の住所フィールドを作成
kashiwa['住所'] = ''

In [60]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 2577 to 2996
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   罪名             420 non-null    object 
 1   手口             420 non-null    object 
 2   管轄警察署（発生地）     420 non-null    object 
 3   管轄交番・駐在所（発生地）  420 non-null    object 
 4   市区町村コード（発生地）   420 non-null    float64
 5   都道府県（発生地）      420 non-null    object 
 6   市区町村（発生地）      420 non-null    object 
 7   町丁目（発生地）       420 non-null    object 
 8   発生年月日（始期）      420 non-null    object 
 9   発生時（始期）        420 non-null    object 
 10  発生場所           420 non-null    object 
 11  発生場所の詳細        420 non-null    object 
 12  被害者の年齢         420 non-null    object 
 13  被害者の職業         420 non-null    object 
 14  施錠関係           420 non-null    object 
 15  住所             420 non-null    object 
dtypes: float64(1), object(15)
memory usage: 55.8+ KB


In [61]:
# 空だがfloatとしてフィールドを作成
kashiwa['lat'] = pd.Series(dtype=float)
kashiwa['lon'] = pd.Series(dtype=float)

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/concat.png" width=600>

↑で作った住所フィールドに次の 3 つのフィールドを連結します。

1. 都道府県（発生地）
1. 市区町村（発生地）
1. 町丁目（発生地）

In [62]:
# 住所フィールドを作成
kashiwa['住所'] = kashiwa['都道府県（発生地）']+kashiwa['市区町村（発生地）']+kashiwa['町丁目（発生地）']

In [63]:
# random sampleで上手く行ったかどうかをチェック
kashiwa[['都道府県（発生地）','市区町村（発生地）','町丁目（発生地）','住所']].sample(5)


,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,住所
2985,千葉県,柏市,高柳,千葉県柏市高柳
2608,千葉県,柏市,根戸,千葉県柏市根戸
2863,千葉県,柏市,若柴,千葉県柏市若柴
2972,千葉県,柏市,豊四季,千葉県柏市豊四季
2984,千葉県,柏市,高柳,千葉県柏市高柳


これで準備は整えました。`kashiwa` のデータフレームを for loop に入れて住所を一つずつジオコーティングしよう。

この作業は数分かかるので注意。千件以上のデータフレームはなるべく避けよう。

In [64]:
# kashiwaデータフレームをループ
for i,row in kashiwa.iterrows():

    # ジオコーティング成功
    if geocode(row['住所']) != False:
        # 座標を変数に
        lon = geocode(row['住所'])[0]
        lat = geocode(row['住所'])[1]

        # データフレームに値をインプット        
        kashiwa.loc[i,'lon'] = lon
        kashiwa.loc[i,'lat'] = lat

        # 結果をprint
        print(row['住所'],lon,lat)
    
    # ジオコーティング失敗
    else:
        print(row['住所'],'ジオコーティング失敗')
        continue

千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷南町 ジオコーティング失敗
千葉県柏市今谷南町 ジオコーティング失敗
千葉県柏市今谷南町 ジオコーティング失敗
千葉県柏市酒井根 139.957825 35.81815
千葉県柏市酒井根２丁目 139.967194 35.821594
千葉県柏市酒井根２丁目 139.967194 35.821594
千葉県柏市酒井根４丁目 139.960007 35.820488
千葉県柏市酒井根４丁目 139.960007 35.820488
千葉県柏市酒井根４丁目 139.960007 35.820488
千葉県柏市中新宿１丁目 139.943588 35.834564
千葉県柏市光ケ丘２丁目 139.951508 35.830315
千葉県柏市光ケ丘団地 ジオコーティング失敗
千葉県柏市光ケ丘団地 ジオコーティング失敗
千葉県柏市光ケ丘団地 ジオコーティング失敗
千葉県柏市光ケ丘団地 ジオコーティング失敗
千葉県柏市光ケ丘団地 ジオコーティング失敗
千葉県柏市東中新宿１丁目 ジオコーティング失敗
千葉県柏市柏 139.986511 35.864445
千葉県柏市柏 139.986511 35.864445
千葉県柏市北柏１丁目 139.985413 35.874477
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏３丁目 139.987793 35.872192
千葉県柏市宿連寺 139.985229 35.887501
千葉県柏市根戸 139.98645 35.88113
千葉県柏市根戸 139.98645 35.88113
千葉県柏市根戸 139.98645 35.88113
千葉県柏市根戸 139.98645 35.88113
千葉県柏市根戸 139.98645 35.88113
千葉県柏市根戸 139.98645 35.88113
千葉県柏市根戸 139.9

In [65]:
# check your output, show address and lat/lon for 5 samples
kashiwa[['住所','lat','lon']].sample(5)


,住所,lat,lon
2739,千葉県柏市つくしが丘１丁目,NaN,NaN
2726,千葉県柏市大津ケ丘１丁目,35.843250,140.003662
2624,千葉県柏市根戸,35.881130,139.986450
2588,千葉県柏市酒井根２丁目,35.821594,139.967194
2597,千葉県柏市光ケ丘団地,NaN,NaN


In [66]:
# geocode出来なかったデータ、latがNaNを削除
kashiwa = kashiwa[kashiwa['lat'].notna()]


## Let's map it!

待ってました、いよいよマップタイム！

In [67]:
# 全データの中央座標
center_lat = kashiwa.lat.mean()
center_lon = kashiwa.lon.mean() 

In [68]:
# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)
 
# kashiwaのデータフレームをループしてマーカーを作る
for index, row in kashiwa.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m


## Heatmap

Foliumのエキストラ機能としてヒートマップが作れる！では、やって見よう。

まずはデータをヒートマップが必要とする形式を作る。

In [69]:
# make a list of lat/lon's
heatmap_lat_lon = kashiwa[['lat','lon']].values.tolist()

# view the list
heatmap_lat_lon

[[35.841019, 139.956406],
 [35.841019, 139.956406],
 [35.841019, 139.956406],
 [35.841019, 139.956406],
 [35.841019, 139.956406],
 [35.841019, 139.956406],
 [35.81815, 139.957825],
 [35.821594, 139.967194],
 [35.821594, 139.967194],
 [35.820488, 139.960007],
 [35.820488, 139.960007],
 [35.820488, 139.960007],
 [35.834564, 139.943588],
 [35.830315, 139.951508],
 [35.864445, 139.986511],
 [35.864445, 139.986511],
 [35.874477, 139.985413],
 [35.872643, 139.989914],
 [35.872192, 139.987793],
 [35.887501, 139.985229],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 

In [70]:
from folium.plugins import HeatMap

# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12,
               tiles='cartodbdark_matter')

# add the heatmap
HeatMap(
    data = heatmap_lat_lon,
    radius=15,    
).add_to(m)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)

# kashiwaのデータフレームをループしてマーカーを作る
for index, row in kashiwa.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m

# 今日の課題

❶ まずは以下のWeek 8 Surveyを提出する。

❷ 今週はcatch up weekです。かなり宿題提出ができてないので、今週で必ず過去の宿題を提出するように。

❸ ボーナス課題（50点）
興味のあるトピックを見つけて、次の内容を説明するノートブックを作成する：

- title (markdown)
- explanation of topic (markdown)
- data (markdown and python)
- analysis (markdown and python)
- mapping (markdown and python)
- 感想 (markdown)

GitHubにコミットし、リンクを提出
